In [1]:
# 0 - legitimate
# 1 - phishing
# 2 - suspicious

In [2]:
# packages
import pandas as pd
from urllib.parse import urlparse,urlencode
import re
from bs4 import BeautifulSoup
import requests
import whois
import urllib.request
from datetime import datetime
import time
import socket
from urllib.error import HTTPError
from cython.parallel import prange
from IPython.display import HTML
import csv, sqlite3, os
from datetime import datetime, timedelta

In [3]:
HTML('''<script>
function code_toggle() {
 if (code_show){
 $('div.input').hide('500');
 $('div.prompt').hide();
 } else {
 $('div.input').show('500');
 $('div.prompt').show();
 }
 code_show = !code_show
} 
$( document ).ready(function(){
    code_show=true;
    $('div.input').show()
    $('div.prompt').show();
  });
</script>
<font size="4" color='red'>The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.</font>''')

In [4]:
strg=r"\AppData\Local\Google\Chrome\User Data\Default"
data_path = os.path.expanduser('~')+strg
files = os.listdir(data_path)
history_db = os.path.join(data_path, 'history')

c = sqlite3.connect(history_db,timeout=10)
cursor = c.cursor()
select_statement = "SELECT urls.url FROM urls;"
cursor.execute(select_statement)

results = cursor.fetchall()
print(type(results))

i=2
cnt=1
fhand=open(r"C:\Users\admin\Desktop\laptop\raw_datasets\real-time-data.txt","w")
while i<len(results):
    s=str(results[i])
    snew=s[2:-3]
    print(str(cnt)+". "+snew)
    print("\n")
    fhand.write(snew+"\n")
    i=i+1
    cnt=cnt+1
fhand.close()

<class 'list'>
1. http://123movieonlinez.in.websiteoutlook.com/


2. http://allmarketing.go2cloud.org/aff_c?offer_id=1421&aff_id=1323


3. http://constintptr.com/1675303/


4. http://constintptr.com/?z=1675303


5. http://constintptr.com/afu.php?zoneid=1675303&var=1675303&rid=wfxzsvAkbQDjdtH2xjZy_Q%3D%3D


6. http://docs.anaconda.com/anaconda/user-guide/getting-started/


7. http://download.filmfanatic.com/index.jhtml?partner=Z1xpu167&pkw=default&adfi=&adti=kwd-0&adm=&adn=d&add=c&adc=274376799088&adt=&ada=&adap=none&adp=123movieonlinez.in.websiteoutlook.com&gclid=EAIaIQobChMIttHRve224QIVDmeOCh3UQwQIEAEYASAAEgJzQfD_BwE


8. http://engine.spotscenered.info/Redirect.eng?MediaSegmentId=0&TempMedia=OIlfW_jCEkUaH1cXC_hMdji0TYb4WjDhW7poduFS6QlwtK9wFMWzKKCuTtakLUQHjL3m8O90AAusOVTVx0TN6vCn2iF0UtZ0oFV-t-9cKd027HrNeiBApn4wbGPJyV745CHBv9EqwgPV4h7grPFxowzGeV98RwailOsvJJ5W7xSWqEoQMmY46E1GGIG9VvIbWqTuvMNUO1qr2XDTmuOdXpJy2PY1HZS_mZNIvl72GRZaoWDLZM3CIDQCy37eTqug9fQJTyncH8MqVVVOQM7tWV0SJuVjavjjYn49RTZLg

In [5]:
from datetime import datetime
itime = datetime.now()
for i in prange(0,10000):
    pass
ftime = datetime.now()
print(ftime-itime)

0:00:00


In [6]:
#computer_raw_data=pd.read_csv("H:/CDAC_PROJECT/PhishingWebsiteDetection/legitimate_urls.txt",header=None,names=["urls"])
#computer_raw_data.head(10)


In [7]:
computer_raw_data=pd.read_csv("raw_datasets/real-time-data.txt",header=None,names=['urls'])
#computer_raw_data

In [8]:
#rows = len(computer_raw_data)

In [9]:
#computer_raw_data["urls"][0]

In [10]:
cd = None
class FeatureExtraction:
    def __init__(self):
        pass
    
    def getProtocol(self,url):
        return urlparse(url).scheme
    
    def getDomain(self,url):
        return urlparse(url).netloc
    
    def getPath(self,url):
        return urlparse(url).path
    

    
    def long_url(self,url):
        """This function is defined in order to differntiate website based on the length of the URL"""
        if len(url) < 54:
            return 0            # legitimate
        elif len(url) >= 54 and len(url) <= 75:
            return 2            # suspicious
        else:
            return 1            # phishing
    
    def have_at_symbol(self,url):
        """This function is used to check whether the URL contains @ symbol or not"""
        if "@" in url:
            return 1            # phishing
        else:
            return 0            # legitimate
    
    def redirection(self,url):
        """If the url has symbol(//) after protocol then such URL is to be classified as phishing """
        if "//" in urlparse(url).path:
            return 1            # phishing
        else:
            return 0            # legitimate
        
    def prefix_suffix_separation(self,url):
        """If the domain has '-' symbol then it is considered as phishing site"""
        if "-" in urlparse(url).netloc:
            return 1            # phishing
        else:
            return 0            # legitimate
        
    def sub_domains(self,url):
        """If the url has more than 3 dots then it is a phishing"""
        if url.count(".") < 3:
            return 0            # legitimate
        elif url.count(".") == 3:
            return 2            # suspicious
        else:
            return 1            # phishing
        
    def shortening_service(self,url):
        """Tiny URL -> phishing otherwise legitimate"""
        match=re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                    'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                    'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                    'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                    'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                    'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                    'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|tr\.im|link\.zip\.net',url)
        if match:
            return 1               # phishing
        else:
            return 0               # legitimate
        
 
    
    def web_traffic(self,url):
        try:
            rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
        except TypeError:
            return 1
        except HTTPError:
            return 2
        rank= int(rank)
        if (rank<100000):
            return 0
        else:
            return 2
        
    def domain_registration_length(self,url):
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
        except:
            dns = 1
        
        if dns == 1:
            return 1      #phishing
        else:
            expiration_date = domain_name.expiration_date
            today = time.strftime('%Y-%m-%d')
            today = datetime.strptime(today, '%Y-%m-%d')
            if expiration_date is None:
                return 1
            elif type(expiration_date) is list or type(today) is list :
                return 2     #If it is a type of list then we can't select a single value from list. So,it is regarded as suspected website  
            else:
                creation_date = domain_name.creation_date
                expiration_date = domain_name.expiration_date
                if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
                    try:
                        creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
                        expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
                    except:
                        return 2
                registration_length = abs((expiration_date - today).days)
                if registration_length / 365 <= 1:
                    return 1 #phishing
                else:
                    return 0 # legitimate
            
    def age_domain(self,url):
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
        except:
            dns = 1
        
        if dns == 1:
            return 1
        else:
            creation_date = domain_name.creation_date
            expiration_date = domain_name.expiration_date
            if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
                try:
                    creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
                    expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
                except:
                    return 2
            if ((expiration_date is None) or (creation_date is None)):
                return 1
            elif ((type(expiration_date) is list) or (type(creation_date) is list)):
                return 2
            else:
                ageofdomain = abs((expiration_date - creation_date).days)
                if ((ageofdomain/30) < 6):
                    return 1
                else:
                    return 0
     
    
    def dns_record(self,url):
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
            #rint(domain_name)
        except:
            dns = 1
        
        if dns == 1:
            return 1
        else:
            return 0
        
   
    def statistical_report(self,url):
        hostname = url
        h = [(x.start(0), x.end(0)) for x in re.finditer('https://|http://|www.|https://www.|http://www.', hostname)]
        z = int(len(h))
        if z != 0:
            y = h[0][1]
            hostname = hostname[y:]
            h = [(x.start(0), x.end(0)) for x in re.finditer('/', hostname)]
            z = int(len(h))
            if z != 0:
                hostname = hostname[:h[0][0]]
        url_match=re.search('at\.ua|usa\.cc|baltazarpresentes\.com\.br|pe\.hu|esy\.es|hol\.es|sweddy\.com|myjino\.ru|96\.lt|ow\.ly',url)
        try:
            ip_address = socket.gethostbyname(hostname)
            ip_match=re.search('146\.112\.61\.108|213\.174\.157\.151|121\.50\.168\.88|192\.185\.217\.116|78\.46\.211\.158|181\.174\.165\.13|46\.242\.145\.103|121\.50\.168\.40|83\.125\.22\.219|46\.242\.145\.98|107\.151\.148\.44|107\.151\.148\.107|64\.70\.19\.203|199\.184\.144\.27|107\.151\.148\.108|107\.151\.148\.109|119\.28\.52\.61|54\.83\.43\.69|52\.69\.166\.231|216\.58\.192\.225|118\.184\.25\.86|67\.208\.74\.71|23\.253\.126\.58|104\.239\.157\.210|175\.126\.123\.219|141\.8\.224\.221|10\.10\.10\.10|43\.229\.108\.32|103\.232\.215\.140|69\.172\.201\.153|216\.218\.185\.162|54\.225\.104\.146|103\.243\.24\.98|199\.59\.243\.120|31\.170\.160\.61|213\.19\.128\.77|62\.113\.226\.131|208\.100\.26\.234|195\.16\.127\.102|195\.16\.127\.157|34\.196\.13\.28|103\.224\.212\.222|172\.217\.4\.225|54\.72\.9\.51|192\.64\.147\.141|198\.200\.56\.183|23\.253\.164\.103|52\.48\.191\.26|52\.214\.197\.72|87\.98\.255\.18|209\.99\.17\.27|216\.38\.62\.18|104\.130\.124\.96|47\.89\.58\.141|78\.46\.211\.158|54\.86\.225\.156|54\.82\.156\.19|37\.157\.192\.102|204\.11\.56\.48|110\.34\.231\.42',ip_address)  
        except:
            return 1

        if url_match:
            return 1
        else:
            return 0
        
    def https_token(self,url):
        match=re.search('https://|http://',url)
        try:
            if match.start(0)==0 and match.start(0) is not None:
                url=url[match.end(0):]
                match=re.search('http|https',url)
                if match:
                    return 1
                else:
                    return 0
        except:
            return 1

    def havingIP(self,url):
        """If the domain part has IP then it is phishing otherwise legitimate"""
        match=re.search('(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  #IPv4
                    '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)'  #IPv4 in hexadecimal
                    '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}',url)     #Ipv6
        if match:
            #print match.group()
            return 1            # phishing
        else:
            #print 'No matching pattern found'
            return 0            # legitimate


In [11]:
#features
protocol = []
domain = []
path = []
having_ip = []
len_url = []
having_at_symbol = []
redirection_symbol = []
prefix_suffix_separation = []
sub_domains = []
tiny_url = []
abnormal_url = []
web_traffic = []
domain_registration_length = []
dns_record = []
statistical_report = []
age_domain = []
http_tokens = []
#google_index = []

### 0 - legitimate
### 1 - phishing
### 2 - suspicious

In [ ]:
# object creation
fe = FeatureExtraction()
rows = len(computer_raw_data["urls"])

for i in range(0,rows):
    url=computer_raw_data["urls"][i]
    print(i ),print(url)
    protocol.append(fe.getProtocol(url))
    path.append(fe.getPath(url))
    domain.append(fe.getDomain(url))
    having_ip.append(fe.havingIP(url))
    len_url.append(fe.long_url(url))
    having_at_symbol.append(fe.have_at_symbol(url))
    redirection_symbol.append(fe.redirection(url))
    prefix_suffix_separation.append(fe.prefix_suffix_separation(url))
    sub_domains.append(fe.sub_domains(url))
    tiny_url.append(fe.shortening_service(url))
    web_traffic.append(fe.web_traffic(url))
    domain_registration_length.append(fe.domain_registration_length(url))
    dns_record.append(fe.dns_record(url))
    statistical_report.append(fe.statistical_report(url))
    age_domain.append(fe.age_domain(url))
    http_tokens.append(fe.https_token(url))
    #google_index.append(fe.google_index(url))
    #abnormal_url.append(fe.abnormal_url(url))
    
#print(domain)
#print(protocol)
#print(ip)
#print(len_url)
#print(google_index)
#print(abnormal_url)
#print(web_traffic)
#print(domain_registration_length)
#print(age_domain)
#print(dns_record)
#print(statistical_report)

0
http://123movieonlinez.in.websiteoutlook.com/
1
http://allmarketing.go2cloud.org/aff_c?offer_id=1421&aff_id=1323
2
http://constintptr.com/1675303/
3
http://constintptr.com/?z=1675303
4
http://constintptr.com/afu.php?zoneid=1675303&var=1675303&rid=wfxzsvAkbQDjdtH2xjZy_Q%3D%3D
5
http://docs.anaconda.com/anaconda/user-guide/getting-started/
6
http://download.filmfanatic.com/index.jhtml?partner=Z1xpu167&pkw=default&adfi=&adti=kwd-0&adm=&adn=d&add=c&adc=274376799088&adt=&ada=&adap=none&adp=123movieonlinez.in.websiteoutlook.com&gclid=EAIaIQobChMIttHRve224QIVDmeOCh3UQwQIEAEYASAAEgJzQfD_BwE
7
http://engine.spotscenered.info/Redirect.eng?MediaSegmentId=0&TempMedia=OIlfW_jCEkUaH1cXC_hMdji0TYb4WjDhW7poduFS6QlwtK9wFMWzKKCuTtakLUQHjL3m8O90AAusOVTVx0TN6vCn2iF0UtZ0oFV-t-9cKd027HrNeiBApn4wbGPJyV745CHBv9EqwgPV4h7grPFxowzGeV98RwailOsvJJ5W7xSWqEoQMmY46E1GGIG9VvIbWqTuvMNUO1qr2XDTmuOdXpJy2PY1HZS_mZNIvl72GRZaoWDLZM3CIDQCy37eTqug9fQJTyncH8MqVVVOQM7tWV0SJuVjavjjYn49RTZLg_K8P7RcEDclT7u8zJhnxIhCZvNZr5-YdfA6Yb

62
https://accounts.google.co.in/accounts/SetSID?ssdc=1&sidt=ALWU2csXTxDY9INcVAhyrjGD1UnaqzxvPlzyQv96qZMyo5z5C0dDTEBpNIcABxyTUT2RzATlyqSB4S9JT8B0%2BDk4W%2BU37wTXxq3BXYoAqiGpVo4LxzCiJFalOREaWXUQ1NbyuenCMh8iTWcyiX8FhxEGyoHA31Ca%2BaDvqgmfwfpD4DbG57E8iCepX3mlUtnXNvGyClgsA4dPf1ftaaIA2%2BcDPr%2Bn/edLaWfFnsUHPuhu/yngKHHS%2Bpk2xud1rvBdLWsbsOw31Z1rA1Of4bUq%2BeeKrGQd6ekqJl%2BuAwxSDuZF%2BchgHWgNQNc%3D&continue=https://mail.google.com/mail/
63
https://accounts.google.co.in/accounts/SetSID?ssdc=1&sidt=ALWU2ct4cxtyhhbU883jfUcxoe180EXmpcH0mfrmP%2B0gzndGOhWqiuogbBr1VLNPIyDyDIbnD0LhrsTwExLYAo3mXKD4oRKWqKPurK/N30NavJcd0zJkRWdUyZ6ewnEVOJjFRKYBW9Q/bTkz/2Hf1wUUjesnAl4JoJzrERIYpLESGWnKI6cXGZTj%2B7RnWvWIrcivBl8j6aq2ZBn2BUnyRH6CVfe1TeBw4eOwPOipo3gxtOPgh5Kd3r62pjchpg9FiAtRWoEYLw8pH%2Boj%2BqkULfhrySmuvInj3dDyStVBufn/ATzzruT7jzg%3D&continue=https://mail.google.com/mail/
64
https://accounts.google.co.in/accounts/SetSID?ssdc=1&sidt=ALWU2ctYP9B2UvuG1XqlfW/1UzL89OoMj8o/rqOCW%2BjHwmvxrOB8lQThvlslMDCwpa%2BESVdrZIw/2%2

94
https://accounts.google.com/ServiceLogin?continue=https%3A%2F%2Fmail.google.com%2Fmail%2F&osid=1&service=mail&ss=1&ltmpl=default&rm=false
95
https://accounts.google.com/ServiceLogin?service=mail&passive=true&rm=false&continue=https://mail.google.com/mail/&ss=1&scc=1&ltmpl=default&ltmplcache=2&emr=1&osid=1#
96
https://accounts.google.com/_/back?backstep=1
97
https://accounts.google.com/info/unknownerror?service=mail&passive=true&rm=false&continue=https%3A%2F%2Fmail.google.com%2Fmail%2F&ss=1&scc=1&ltmpl=default&ltmplcache=2&emr=1&osid=1&flowName=GlifWebSignIn&flowEntry=AddSession
98
https://accounts.google.com/signin/v2/sl/pwd?continue=https%3A%2F%2Fmail.google.com%2Fmail%2F&osid=1&service=mail&ss=1&ltmpl=default&rm=false&flowName=GlifWebSignIn&flowEntry=AddSession&cid=0&navigationDirection=forward
99
https://accounts.google.com/signin/v2/sl/pwd?service=mail&passive=true&rm=false&continue=https%3A%2F%2Fmail.google.com%2Fmail%2F&ss=1&scc=1&ltmpl=default&ltmplcache=2&emr=1&osid=1&flowNa

131
https://docs.anaconda.com/anaconda/packages/old-pkg-lists/5.3.0/py3.6_win-64/
132
https://docs.anaconda.com/anaconda/user-guide/getting-started
133
https://dolohen.com/?ip=adf8e688c57e41a7451fabdf674ab666&pbk3=0b60730543b3f62dcaf956344145d3136676077723675921795&r=%2Foc%2Fhan&uuid=d3c13b57-decb-4ef8-a54b-c2f8c6a06cab&co=0&rf=1&zoneid=2069403&xref=Mm1vdmllcnVsei5zdA&fs=0&cf=0&sw=1440&sh=900&sah=860&wx=0&wy=0&ww=1440&wh=860&cw=1423&wiw=1440&wih=757&wfc=0&pl=https%3A%2F%2F2movierulz.st%2F&drf=https%3A%2F%2Fwww.google.com%2F&np=1&pt=0&nb=1&ng=1&ix=0&nw=0
134
https://dolohen.com/?ip=adf8e688c57e41a7451fabdf674ab666&pbk3=0b60730543b3f62dcaf956344145d3136676077723675921795&r=%2Foc%2Fhan&uuid=d3c13b57-decb-4ef8-a54b-c2f8c6a06cab&co=0&rf=1&zoneid=2069403&xref=Mm1vdmllcnVsei5zdA&fs=0&cf=0&sw=1440&sh=900&sah=860&wx=0&wy=0&ww=1440&wh=860&cw=1423&wiw=1440&wih=757&wfc=0&pl=https%3A%2F%2F2movierulz.st%2Fjamba-lakidi-pamba-2018-hdrip-telugu-full-movie-watch-online-free%2F&drf=https%3A%2F%2F2movieru

161
https://mail.google.com/accounts/SetOSID?authuser=0&continue=https%3A%2F%2Faccounts.youtube.com%2Faccounts%2FSetSID%3Fssdc%3D1%26sidt%3DALWU2cslkughvuKUQL6SEBMq%252FX4d0hrPhxjDKxy1vSTNr8eq%252FPoy2GHfKt5G1YLhy2NbvhVVVH7gD757v7zA1qUfv7FahcwADgYW%252FO2j8t4oZ8ryJPFsD%252FE4g9vhJjo9i3vSqaR%252Bhu6KHrW87cph%252Ftc4Lg%252BOxVDgdW%252BfKweD8frX3uRSC2I3FvMjfN79CtimICUPE%252BPi2fcHjk4bZ%252BXSMvJPZcYh%252BkPT2EQCcO2nYan6xOt%252Fu54R6LTC1Ye5l%252Fjj3KvZS0sKWyW8VmFqVMdQ66tA10zQCUvIgPSu%252FgC1y0UntRxZb0YUkrQ%253D%26continue%3Dhttps%253A%252F%252Fmail.google.com%252Fmail%252F%26dbus%3DIN&osidt=ALWU2ctIhnwVHti1sLp_0doj6P3thBMviCm0IT1yhzj8TVQvd7QzE3qlSSvergWNEvzooAVuEddYfJ3f8mukwBgkIRrm-7rQIwlecOzrvO-dZFCOVVuHtdo-qheTw_n8gNP8IU3t6IS6
162
https://mail.google.com/accounts/SetOSID?authuser=0&continue=https%3A%2F%2Faccounts.youtube.com%2Faccounts%2FSetSID%3Fssdc%3D1%26sidt%3DALWU2csnk3C6fN2nFa6n5eVzim8fB2NexOR1RZpH9slmNlQ%252Fxy1%252Fm3dRupjHBm6Vj8hw%252BKfLvpgsRbfGNTZZOYQDXb3ZWre7FRl%252Fecfn0v%25

173
https://mail.google.com/accounts/SetOSID?authuser=0&continue=https%3A%2F%2Fmail.google.com%2Fmail%2F&osidt=ALWU2csopUOMw6-_em7dfVwJXOzyclEJjZrVDEY9iVAip2sIiBGC1XooCrJwgEyKtxwc7cXrf9d7Oe2qrB6exoDa4kf8FT782tKAk3yikA6XBuGeuAeQPTcK4TyU1Pmi2Fgfily51sLu
174
https://mail.google.com/mail
175
https://mail.google.com/mail/
176
https://mail.google.com/mail/u/0/
177
https://mail.google.com/mail/u/0/#inbox
178
https://mail.google.com/mail/u/0/#inbox/DmwnWrRpclWbsCFhWDtXbkmDrnhQHRtVVrBRLRtqhRPZntlpzGwvFKMQSkFMFDwmRKTZDFrntLGv
179
https://mail.google.com/mail/u/0/#inbox/WhctKJVJkvpHsGkhlPrfqCBCHgJpXbMLkpMMWWMfZRgFmjSKgmKpcSNQPjXSRFTTCpBkzZL
180
https://mail.google.com/mail/u/0/#inbox/WhctKJVJkvpHsGkhlPrfqCBCHgJpXbMLkpMMWWMfZRgFmjSKgmKpcSNQPjXSRFTTCpBkzZL?compose=DXDwSWxDjFhLLsDFZXRFNZKtMTtMwfLxqPpbtdcmVMpNSrTVHMQQFlrpPFjstsXXDXwgfNvpTHVpPFmDQgzxxJzZhVwHxGgnrqsjSwnTNXtvLvqjpdsCTnNq
181
https://mail.google.com/mail/u/0/#inbox/WhctKJVJkvpHsGkhlPrfqCBCHgJpXbMLkpMMWWMfZRgFmjSKgmKpcSNQPjXSRFTTCpBkzZL?c

231
https://serve.popads.net/s?cid=6129645&iuid=1481440928&ts=1553530521&ps=2637296495&pw=321&pl=%21d3Zoe8WWVgwxYc%2Fp%2FRBVLnD4KTrf0qKt5pwto1zrrhzFKN4xCxxNOHEc8IjXstdqFVspul4lJOPc8L2OJnBWYYx1YUGVbUap3WF8lHCO6aNebWtwrBDlp%2FO%2BIdj6webg%2FvP3bPbsV4nDsDVYvsno9%2BR4j%2BWyhXg7s30hjpd8RKqFg98H9rSPYJ7Rcw%2FphEzi%2FEa9L5VPIohYEb1EzPkvzXIk9QdwCD9wa1KuQgRBkPrUPpwS4YNXFW%2FcLHahpjCVmF5krUbYhYGDNv%2BpF5rv%2BBC%2BHGPHAzfB4kx%2Byes5gdzGc8EP9eSEx37%2BjHoOTMh3
232
https://stackoverflow.com/questions/11556781/can-we-get-browser-history-by-using-javascript
233
https://stackoverflow.com/questions/1347791/unicode-error-unicodeescape-codec-cant-decode-bytes-cannot-open-text-file
234
https://stackoverflow.com/questions/151026/how-do-i-unlock-a-sqlite-database
235
https://stackoverflow.com/questions/15653688/preserving-column-order-in-python-pandas-dataframe
236
https://stackoverflow.com/questions/24773109/how-to-downgrade-the-installed-version-of-pip-on-windows
237
https://stackoverflow.com/questions/2561

282
https://www.google.com/search?q=bnmit&oq=bnmit&aqs=chrome..69i57j0l5.1096j1j7&sourceid=chrome&ie=UTF-8
283
https://www.google.com/search?q=bnmit+cms&oq=bnmit+cms&aqs=chrome..69i57j69i59.1890j1j7&sourceid=chrome&ie=UTF-8
284
https://www.google.com/search?q=bnmit+cms+dhi&oq=bnmit+cms&aqs=chrome.0.69i59j69i57.2287j0j9&sourceid=chrome&ie=UTF-8
285
https://www.google.com/search?q=bnmit+cms+dhi&oq=bnmit+cms&aqs=chrome.0.69i59j69i57.2642j0j9&sourceid=chrome&ie=UTF-8
286
https://www.google.com/search?q=bnmit+cms+dhi&oq=bnmit+cms+dhi&aqs=chrome..69i57.25382j0j4&sourceid=chrome&ie=UTF-8
287
https://www.google.com/search?q=bnmit+dhi&oq=bnmit+dhi&aqs=chrome..69i57.2186j0j7&sourceid=chrome&ie=UTF-8
288
https://www.google.com/search?q=can+jupyter+notebook+be+run+without+chrome%3F&oq=can+jupyter+notebook+be+run+without+chrome%3F&aqs=chrome..69i57.15057j1j7&sourceid=chrome&ie=UTF-8
289
https://www.google.com/search?q=check+anaconda+version&oq=checkiong+anaconda+ver&aqs=chrome.1.69i57j0l5.7391j0j7&

335
https://www.google.com/url?sa=i&source=images&cd=&ved=2ahUKEwjFoYOx9JfiAhWILo8KHSNnByIQjRx6BAgBEAU&url=https%3A%2F%2Fgrowthhackers.com%2Fgrowth-studies%2Fairbnb&psig=AOvVaw2dFTemgtpaVt15TxIzOJZt&ust=1557816378325468
336
https://www.googleadservices.com/pagead/aclk?sa=L&ai=CDonRYTCmXPacAY7OuQTUh5FA3fKe6lXMmNW7pgeTvL3VsA0QASDzu8snKAZg5crlg7QOkAECoAHitrTPA8gBAagDAcgDAqoE2AFP0IL8XXjCBe1MACKn4X4nga6ftk3NPwAc0n37r19zVAqx0lClLvLCin6XZAOJuonN7F8Wghjol1oW_rDf1hUmwybRtHpApFn6P6aEUG_loqFbuYb02pa0o85ryV9LVnqnzUSpYRIm2sDAS3zVIJoL0GdcuYeU3qISXkfMLyD4qDhyXRn5nn8liUsf96WYtbuxiA-LExThv3-NatgCtaRa9XJLKic3FEaIythNHAVLqV3IoVhIeyHmBZfbywLAY6BkDJFYzMumz7kR7sUIHF1BBuQeTU2LWF-AB4bJyzCoB47OG6gH1ckbqAeB1BuoB-DTG6gHqAaoB7oGqAfZyxuoB8_MG6gHpr4b2AcB0ggHCIBhEAEYArEJ1QZwitsx7Z2ACgHYEww&ae=1&num=1&cid=CAASEuRo9ijpQurZEMpKtq9e1BoVpA&sig=AOD64_3A3MCcDrzYhk1hXfqSkPjQkr402g&client=ca-pub-1354500165457876&adurl=http://download.filmfanatic.com/index.jhtml%3Fpartner%3DZ1xpu167%26pkw%3Ddefault%26adfi%3D%26adti%3Dkwd-0%26

In [ ]:
label = []
for i in range(0,rows):
    label.append(1)

In [ ]:
d={'Domain':pd.Series(domain),
   'Having_@_symbol':pd.Series(having_at_symbol),
   'Having_IP':pd.Series(having_ip),
   'Path':pd.Series(path),
   'Prefix_suffix_separation':pd.Series(prefix_suffix_separation),
   'Protocol':pd.Series(protocol),
   'Redirection_//_symbol':pd.Series(redirection_symbol),
   'Sub_domains':pd.Series(sub_domains),
   'URL_Length':pd.Series(len_url),
   'age_domain':pd.Series(age_domain),
   'dns_record':pd.Series(dns_record),
   'domain_registration_length':pd.Series(domain_registration_length),
   'http_tokens':pd.Series(http_tokens),
   'label':pd.Series(label),
   'statistical_report':pd.Series(statistical_report),
   'tiny_url':pd.Series(tiny_url),
   'web_traffic' : pd.Series(web_traffic)}
data=pd.DataFrame(d)
data

In [ ]:
#data.to_csv("legitimate-urls.csv",index=False,encoding='UTF-8')

In [ ]:
data.to_csv(r"C:\Users\admin\Desktop\laptop\extracted_csv_files\real-time.csv",index=False,encoding='UTF-8')

In [ ]:
#columns=["Domain","Having_@_symbol","Having_IP","Path","Prefix_suffix_separation","Protocol","Redirection_//_symbol","Sub_domains","URL_Length","age_domain","dns_record","domain_registration_length","http_tokens","label","statistical_report","tiny_url","web_traffic"]